## Imports

In [1]:
# pip install --upgrade scikit-learn
#!pip install --upgrade scikit-learn


In [2]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   0.25.3
Sklearn  0.24.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [3]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "gawas2inspire"
    os.environ['KAGGLE_KEY']      = "xxxxxxxxxxxxx"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../Datasets/Tabular/titanic/"  # dont change this

In [4]:
!pwd

/home/simple/ai/strive/git_strive/StriveSchool/d36/01. Robust ML


## Load data

In [5]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [ ]:
df.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [6]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)


# YOUR CODE HERE
df['Title']  = df.Name.str.extract('([A-Za-z]+)\.', expand = False)
df_test['Title'] = df_test.Name.str.extract('([A-Za-z]+)\.', expand = False)

#df['Title']      = df['Name'].map(get_Title_from_Name)
#df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

#get_Title_from_Name.value_counts()
#df['Title']

In [7]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [8]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [9]:
# Use map to apply the prevous dict

# df["Title"] =  
# df_test["Title"] =

# YOUR CODE HERE
df["Title"] =  df['Title'].map(title_dictionary)
df_test["Title"] = df_test['Title'].map(title_dictionary)

In [10]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [11]:
df_test['Ticket'] = df_test.Ticket.str.split().apply(lambda x : 0 if x[:][-1] == 'LINE' else x[:][-1])
df_test.Ticket = df_test.Ticket.values.astype('int64')
#df['Ticket'] = df.Ticket.str.split().apply(lambda x : 0 if x[:][-1] == 'LINE' else x[:][-1])
df_test['Ticket']

PassengerId
892      330911
893      363272
894      240276
895      315154
896     3101298
         ...   
1305       3236
1306      17758
1307    3101262
1308     359309
1309       2668
Name: Ticket, Length: 418, dtype: int64

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Cabin'] 

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [13]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [14]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [15]:
"""
num_preprocessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
])

cat_preporcessing = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro
""";

# YOUR CODE HERE

num_4_treeModels = pipeline.Pipeline(steps=[
     # Some SimpleImputer here
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)), # mean, median
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  # Some SimpleImputer here
  # Some Encoder here. Remember to handle_unknown
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [16]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [17]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [23]:
"""
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":
  "Random Forest":
  "AdaBoost":
  "Skl GBM":
  "Skl HistGBM":
  "XGBoost":
  "LightGBM":
  "CatBoost":
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]
""";


# YOUR CODE HERE
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(),
  "Random Forest": RandomForestClassifier(),
  "AdaBoost": AdaBoostClassifier(),
  "Skl GBM": GradientBoostingClassifier(),
  "Skl HistGBM": HistGradientBoostingClassifier(),
  "XGBoost": XGBClassifier(),
  "LightGBM": LGBMClassifier(),
  "CatBoost": CatBoostClassifier()
}
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [24]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [51]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y, test_size = 0.2, random_state = 0, stratify=y
)
x_train_tree_prepro  = tree_prepro.fit_transform(x_train)
x_val_tree_prepro    = tree_prepro.transform(x_val)
x_train_tree_prepro.shape, x_val_tree_prepro.shape
type(x_train_tree_prepro)

numpy.ndarray

In [47]:
pd.DataFrame(data = x_train_prepro, columns=num_vars+cat_vars_oh).head()

,Pclass,SibSp,Parch,Fare,Age,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_missing,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Title_missing
0,3.0,0.0,0.0,7.6292,29.469823,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,3.0,0.0,0.0,8.0500,29.469823,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,7.7500,29.469823,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,77.9583,51.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,3.0,0.0,0.0,7.7333,21.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [72]:
"""
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    # CODE HERE
)
"""
# YOUR CODE HERE
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y, test_size = 0.2, random_state = 0, stratify=y
)
x_train_tree_prepro  = tree_prepro.fit_transform(x_train)
x_val_tree_prepro    = tree_prepro.transform(x_val)
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

""" 
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    
    # GET PREDICTIONS USING x_val
    pred = # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# YOUR CODE HERE
for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    #model.predict(x_val_tree_prepro)
    model.fit(x_train, y_train)
    #full_model = pipeline.Pipeline([('preprocessor', tree_prepro), ('model', model)])
    
    # GET PREDICTIONS USING x_val
    pred =  model.predict(x_val) # CODE HERE y_pred       = model.predict(x_val_prepro)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              

results_ord = results.sort_values(by=['Accuracy'], ascending=False) #, ignore_index=False)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/home/simple/anaconda3/envs/StriveSchool/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:20:20] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6880411	total: 1.31ms	remaining: 1.31s
1:	learn: 0.6827229	total: 4.65ms	remaining: 2.32s
2:	learn: 0.6780399	total: 11.3ms	remaining: 3.75s
3:	learn: 0.6724704	total: 14.9ms	remaining: 3.71s
4:	learn: 0.6668794	total: 18.3ms	remaining: 3.65s
5:	learn: 0.6611551	total: 20.6ms	remaining: 3.42s
6:	learn: 0.6562879	total: 23ms	remaining: 3.26s
7:	learn: 0.6510608	total: 25.5ms	remaining: 3.16s
8:	learn: 0.6455381	total: 27.6ms	remaining: 3.04s
9:	learn: 0.6402343	total: 29.8ms	remaining: 2.95s
10:	learn: 0.6361685	total: 32.1ms	remaining: 2.89s
11:	learn: 0.6317737	total: 34.5ms	remaining: 2.84s
12:	learn: 0.6287853	total: 35.9ms	remaining: 2.73s
13:	learn: 0.6238341	total: 38.1ms	remaining: 2.68s
14

182:	learn: 0.3960865	total: 552ms	remaining: 2.46s
183:	learn: 0.3956174	total: 556ms	remaining: 2.47s
184:	learn: 0.3952417	total: 563ms	remaining: 2.48s
185:	learn: 0.3949582	total: 570ms	remaining: 2.5s
186:	learn: 0.3945736	total: 583ms	remaining: 2.53s
187:	learn: 0.3945034	total: 587ms	remaining: 2.54s
188:	learn: 0.3943947	total: 589ms	remaining: 2.53s
189:	learn: 0.3941684	total: 593ms	remaining: 2.53s
190:	learn: 0.3938177	total: 607ms	remaining: 2.57s
191:	learn: 0.3935588	total: 618ms	remaining: 2.6s
192:	learn: 0.3930231	total: 632ms	remaining: 2.64s
193:	learn: 0.3927477	total: 635ms	remaining: 2.64s
194:	learn: 0.3921858	total: 651ms	remaining: 2.69s
195:	learn: 0.3920841	total: 661ms	remaining: 2.71s
196:	learn: 0.3915131	total: 679ms	remaining: 2.77s
197:	learn: 0.3911828	total: 695ms	remaining: 2.81s
198:	learn: 0.3907660	total: 697ms	remaining: 2.81s
199:	learn: 0.3905114	total: 714ms	remaining: 2.86s
200:	learn: 0.3902133	total: 722ms	remaining: 2.87s
201:	learn: 0.

346:	learn: 0.3595638	total: 1.29s	remaining: 2.44s
347:	learn: 0.3594012	total: 1.3s	remaining: 2.43s
348:	learn: 0.3592181	total: 1.3s	remaining: 2.42s
349:	learn: 0.3591026	total: 1.31s	remaining: 2.43s
350:	learn: 0.3589473	total: 1.31s	remaining: 2.42s
351:	learn: 0.3587314	total: 1.31s	remaining: 2.42s
352:	learn: 0.3586170	total: 1.32s	remaining: 2.41s
353:	learn: 0.3585510	total: 1.32s	remaining: 2.41s
354:	learn: 0.3585157	total: 1.32s	remaining: 2.41s
355:	learn: 0.3584072	total: 1.33s	remaining: 2.4s
356:	learn: 0.3581799	total: 1.33s	remaining: 2.39s
357:	learn: 0.3581033	total: 1.33s	remaining: 2.39s
358:	learn: 0.3580588	total: 1.35s	remaining: 2.42s
359:	learn: 0.3578773	total: 1.36s	remaining: 2.43s
360:	learn: 0.3576599	total: 1.37s	remaining: 2.43s
361:	learn: 0.3574837	total: 1.38s	remaining: 2.43s
362:	learn: 0.3573570	total: 1.38s	remaining: 2.43s
363:	learn: 0.3572006	total: 1.39s	remaining: 2.42s
364:	learn: 0.3571262	total: 1.39s	remaining: 2.42s
365:	learn: 0.3

533:	learn: 0.3348320	total: 2.01s	remaining: 1.75s
534:	learn: 0.3346332	total: 2.02s	remaining: 1.75s
535:	learn: 0.3343813	total: 2.02s	remaining: 1.75s
536:	learn: 0.3342293	total: 2.03s	remaining: 1.75s
537:	learn: 0.3341680	total: 2.04s	remaining: 1.75s
538:	learn: 0.3340386	total: 2.04s	remaining: 1.75s
539:	learn: 0.3339767	total: 2.04s	remaining: 1.74s
540:	learn: 0.3338754	total: 2.05s	remaining: 1.74s
541:	learn: 0.3337213	total: 2.05s	remaining: 1.73s
542:	learn: 0.3335546	total: 2.06s	remaining: 1.73s
543:	learn: 0.3335402	total: 2.06s	remaining: 1.73s
544:	learn: 0.3335185	total: 2.06s	remaining: 1.72s
545:	learn: 0.3333503	total: 2.07s	remaining: 1.72s
546:	learn: 0.3331943	total: 2.07s	remaining: 1.72s
547:	learn: 0.3331087	total: 2.08s	remaining: 1.71s
548:	learn: 0.3330406	total: 2.08s	remaining: 1.71s
549:	learn: 0.3328758	total: 2.09s	remaining: 1.71s
550:	learn: 0.3327685	total: 2.09s	remaining: 1.7s
551:	learn: 0.3326854	total: 2.09s	remaining: 1.7s
552:	learn: 0.

726:	learn: 0.3134073	total: 2.96s	remaining: 1.11s
727:	learn: 0.3132090	total: 2.96s	remaining: 1.1s
728:	learn: 0.3130155	total: 2.96s	remaining: 1.1s
729:	learn: 0.3128692	total: 2.97s	remaining: 1.1s
730:	learn: 0.3128354	total: 2.97s	remaining: 1.09s
731:	learn: 0.3127650	total: 2.97s	remaining: 1.09s
732:	learn: 0.3126486	total: 2.98s	remaining: 1.08s
733:	learn: 0.3125059	total: 2.98s	remaining: 1.08s
734:	learn: 0.3124863	total: 2.99s	remaining: 1.08s
735:	learn: 0.3122687	total: 2.99s	remaining: 1.07s
736:	learn: 0.3121514	total: 3s	remaining: 1.07s
737:	learn: 0.3120778	total: 3s	remaining: 1.06s
738:	learn: 0.3119290	total: 3s	remaining: 1.06s
739:	learn: 0.3117012	total: 3.01s	remaining: 1.06s
740:	learn: 0.3115161	total: 3.01s	remaining: 1.05s
741:	learn: 0.3114494	total: 3.02s	remaining: 1.05s
742:	learn: 0.3113466	total: 3.02s	remaining: 1.04s
743:	learn: 0.3112423	total: 3.02s	remaining: 1.04s
744:	learn: 0.3112077	total: 3.03s	remaining: 1.03s
745:	learn: 0.3110123	to

905:	learn: 0.2914578	total: 3.68s	remaining: 382ms
906:	learn: 0.2911744	total: 3.68s	remaining: 378ms
907:	learn: 0.2911654	total: 3.68s	remaining: 373ms
908:	learn: 0.2909579	total: 3.69s	remaining: 370ms
909:	learn: 0.2907237	total: 3.7s	remaining: 366ms
910:	learn: 0.2904177	total: 3.7s	remaining: 361ms
911:	learn: 0.2903432	total: 3.7s	remaining: 357ms
912:	learn: 0.2902984	total: 3.71s	remaining: 353ms
913:	learn: 0.2901984	total: 3.71s	remaining: 349ms
914:	learn: 0.2899873	total: 3.71s	remaining: 345ms
915:	learn: 0.2898780	total: 3.72s	remaining: 341ms
916:	learn: 0.2897485	total: 3.72s	remaining: 337ms
917:	learn: 0.2896044	total: 3.72s	remaining: 333ms
918:	learn: 0.2893225	total: 3.73s	remaining: 328ms
919:	learn: 0.2892468	total: 3.73s	remaining: 325ms
920:	learn: 0.2890154	total: 3.74s	remaining: 321ms
921:	learn: 0.2889824	total: 3.74s	remaining: 317ms
922:	learn: 0.2889016	total: 3.75s	remaining: 313ms
923:	learn: 0.2886978	total: 3.75s	remaining: 309ms
924:	learn: 0.2

,Model,Accuracy,Bal Acc.,Time
9,CatBoost,87.1508,85.4941,4.73496
5,Skl GBM,86.0335,83.7747,0.424814
6,Skl HistGBM,84.3575,82.9513,7.32146
4,AdaBoost,83.7989,83.0369,0.468812
7,XGBoost,83.2402,82.5823,2.91131
8,LightGBM,83.2402,82.0422,0.676305
3,Random Forest,82.1229,80.3228,0.722875
2,Extra Trees,80.4469,78.6891,0.711844
1,Decision Tree,77.095,76.502,0.101326


In [73]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [77]:

"""
skf = model_selection.StratifiedKFold(
    # CODE HERE
)
"""
# YOUR CODE HERE
skf = model_selection.StratifiedKFold(
    n_splits=10,
    shuffle=True,
    random_state=0
)
skf.split(x, y)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

"""
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = # CODE HERE model_selection.cross_val_predict(model, x, y, cv=10)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
"""

# YOUR CODE HERE
for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y

    pred = model_selection.cross_val_predict(model, x, y, cv=10)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                            "Accuracy": metrics.accuracy_score(y, pred)*100,
                            "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                            "Time":     total_time},
                            ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False)#, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/home/simple/anaconda3/envs/StriveSchool/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:32:49] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/simple/anaconda3/envs/StriveSchool/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:32:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/simple/anaconda3/envs/StriveSchool/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

361:	learn: 0.3545683	total: 940ms	remaining: 1.66s
362:	learn: 0.3544859	total: 946ms	remaining: 1.66s
363:	learn: 0.3543380	total: 949ms	remaining: 1.66s
364:	learn: 0.3540185	total: 951ms	remaining: 1.66s
365:	learn: 0.3538463	total: 954ms	remaining: 1.65s
366:	learn: 0.3537694	total: 957ms	remaining: 1.65s
367:	learn: 0.3536586	total: 959ms	remaining: 1.65s
368:	learn: 0.3535678	total: 961ms	remaining: 1.64s
369:	learn: 0.3534242	total: 965ms	remaining: 1.64s
370:	learn: 0.3533885	total: 966ms	remaining: 1.64s
371:	learn: 0.3532620	total: 969ms	remaining: 1.64s
372:	learn: 0.3531010	total: 971ms	remaining: 1.63s
373:	learn: 0.3530181	total: 973ms	remaining: 1.63s
374:	learn: 0.3529509	total: 976ms	remaining: 1.63s
375:	learn: 0.3528276	total: 978ms	remaining: 1.62s
376:	learn: 0.3526308	total: 980ms	remaining: 1.62s
377:	learn: 0.3523285	total: 983ms	remaining: 1.62s
378:	learn: 0.3521822	total: 985ms	remaining: 1.61s
379:	learn: 0.3521410	total: 987ms	remaining: 1.61s
380:	learn: 

584:	learn: 0.3289430	total: 1.5s	remaining: 1.07s
585:	learn: 0.3288325	total: 1.51s	remaining: 1.06s
586:	learn: 0.3285260	total: 1.51s	remaining: 1.06s
587:	learn: 0.3283685	total: 1.51s	remaining: 1.06s
588:	learn: 0.3282940	total: 1.51s	remaining: 1.06s
589:	learn: 0.3282068	total: 1.52s	remaining: 1.05s
590:	learn: 0.3280732	total: 1.52s	remaining: 1.05s
591:	learn: 0.3280228	total: 1.52s	remaining: 1.05s
592:	learn: 0.3280122	total: 1.52s	remaining: 1.04s
593:	learn: 0.3279416	total: 1.52s	remaining: 1.04s
594:	learn: 0.3276957	total: 1.53s	remaining: 1.04s
595:	learn: 0.3276023	total: 1.53s	remaining: 1.04s
596:	learn: 0.3275289	total: 1.53s	remaining: 1.03s
597:	learn: 0.3274597	total: 1.53s	remaining: 1.03s
598:	learn: 0.3271911	total: 1.54s	remaining: 1.03s
599:	learn: 0.3271662	total: 1.54s	remaining: 1.02s
600:	learn: 0.3269120	total: 1.54s	remaining: 1.02s
601:	learn: 0.3268925	total: 1.54s	remaining: 1.02s
602:	learn: 0.3267438	total: 1.54s	remaining: 1.02s
603:	learn: 0

746:	learn: 0.3116548	total: 1.88s	remaining: 635ms
747:	learn: 0.3116485	total: 1.88s	remaining: 632ms
748:	learn: 0.3115745	total: 1.88s	remaining: 630ms
749:	learn: 0.3113715	total: 1.88s	remaining: 627ms
750:	learn: 0.3112781	total: 1.88s	remaining: 624ms
751:	learn: 0.3109744	total: 1.89s	remaining: 622ms
752:	learn: 0.3108044	total: 1.89s	remaining: 619ms
753:	learn: 0.3106151	total: 1.89s	remaining: 617ms
754:	learn: 0.3104823	total: 1.89s	remaining: 614ms
755:	learn: 0.3103843	total: 1.89s	remaining: 611ms
756:	learn: 0.3103651	total: 1.9s	remaining: 608ms
757:	learn: 0.3103346	total: 1.9s	remaining: 606ms
758:	learn: 0.3102556	total: 1.9s	remaining: 603ms
759:	learn: 0.3102125	total: 1.9s	remaining: 601ms
760:	learn: 0.3100486	total: 1.9s	remaining: 598ms
761:	learn: 0.3099360	total: 1.91s	remaining: 596ms
762:	learn: 0.3098548	total: 1.91s	remaining: 593ms
763:	learn: 0.3096527	total: 1.91s	remaining: 590ms
764:	learn: 0.3095687	total: 1.91s	remaining: 588ms
765:	learn: 0.309

983:	learn: 0.2854344	total: 2.43s	remaining: 39.6ms
984:	learn: 0.2853805	total: 2.44s	remaining: 37.1ms
985:	learn: 0.2853428	total: 2.44s	remaining: 34.6ms
986:	learn: 0.2851633	total: 2.44s	remaining: 32.1ms
987:	learn: 0.2851039	total: 2.44s	remaining: 29.7ms
988:	learn: 0.2848589	total: 2.44s	remaining: 27.2ms
989:	learn: 0.2846852	total: 2.45s	remaining: 24.7ms
990:	learn: 0.2845743	total: 2.45s	remaining: 22.3ms
991:	learn: 0.2845093	total: 2.45s	remaining: 19.8ms
992:	learn: 0.2844612	total: 2.46s	remaining: 17.3ms
993:	learn: 0.2843518	total: 2.46s	remaining: 14.8ms
994:	learn: 0.2842981	total: 2.46s	remaining: 12.4ms
995:	learn: 0.2842590	total: 2.46s	remaining: 9.89ms
996:	learn: 0.2841682	total: 2.46s	remaining: 7.42ms
997:	learn: 0.2841187	total: 2.47s	remaining: 4.95ms
998:	learn: 0.2838785	total: 2.47s	remaining: 2.47ms
999:	learn: 0.2837866	total: 2.47s	remaining: 0us


,Model,Accuracy,Bal Acc.,Time
9,CatBoost,83.8384,81.5382,29.2362
5,Skl GBM,83.6139,81.5766,2.65344
8,LightGBM,81.4815,79.6808,1.85296
4,AdaBoost,81.257,80.1601,3.31297
7,XGBoost,81.0325,79.7023,3.74224
6,Skl HistGBM,80.9203,79.2254,8.92739
3,Random Forest,80.5836,79.2278,4.032
2,Extra Trees,80.3591,78.77,3.30431
1,Decision Tree,76.9921,76.0378,0.452026


In [78]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [79]:
best_model = tree_classifiers["Skl HistGBM"]  # Skl GBM	83.6139	81.5766	2.65344, almost best accuracy in less time

# Fit best model with all data

# YOUR CODE HERE
best_model.fit(x, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [80]:
test_pred = 0# Get the predictions for x_test

# YOUR CODE HERE
test_pred = best_model.predict(x_test)

In [81]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [82]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [83]:
sub.to_csv("sub.csv")

In [ ]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"